In [5]:
#Import necessary packages
import requests
import pickle
import pandas as pd
import numpy as np
from pymongo import MongoClient
from datetime import date
import time

#Bring in pickled objects
list_of_topics = pickle.load(open('list_of_topics','rb'))

#Open MongoDB connection
client = MongoClient()

#Create database/start connection to databse
db = client.news
articles = db.news

In [14]:
#Create function for API query initial database
def query_news(topic):
    url = "https://free-news.p.rapidapi.com/v1/search"
    querystring = {'q': topic,'lang':'en', 'page_size':'100'}
    headers = {
        'x-rapidapi-key': '780fcaeeb3mshf9210d76d977dc2p1702dbjsn0f8d212f07b3',
        'x-rapidapi-host': 'free-news.p.rapidapi.com'
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response.json()

#Create test workflow to ensure database and functions are working properly
#Create test collection
#test_collection = db.news

#Query API
test_query = query_news('flooding')

#Pull out articles specifically
query_to_insert = list(test_query['articles'])

#Insert articles
db.test_collection.insert_many(query_to_insert)

#Test to ensure articles are contained in database
list(db.test_collection.find({}, {'_id':0, 'title': 1}))

#Pull articles from API
for topic in list_of_topics:
    try:
        working_query = query_news(topic)
        working_query = list(working_query['articles'])
        db.articles.insert_many(working_query, ordered=False)
    except:
        print('No articles found for the following topic:', topic)
    print(db.articles.count(), 'articles added so far')
    time.sleep(1)

["MeToo gave women the power to say 'no'",
 "The #MeToo Movement Gave Women the Power to Say ‘No'",
 'Sexual harassment',
 "Gloria Allred Isn't Worried About What Bill Cosby's Release Means for MeToo",
 'Lecturer wins damages from #MeToo blog that falsely accused him of rape',
 "Casanova and His Female Counterparts, in Benoît Jacquot's Latest",
 'Case registered against Bollywood producer Bhushan Kumar on allegations of rape',
 "Harnessing Sri Lanka's #metoo Moment into a Movement",
 "Cannes falls for 'The Worst Person in the World'",
 "Cannes falls for 'The Worst Person in the World'",
 "Cannes falls for 'The Worst Person in the World'",
 "Cannes falls for 'The Worst Person in the World'",
 "Cannes falls for 'The Worst Person in the World'",
 "Cannes falls for 'The Worst Person in the World'",
 'Partisans are more likely to believe that members of the other party are guilty of sexual misconduct than members of their own party. There is an exception to this: Individuals who support the